# Genetic Agents

Let's win some cartpole!

Genetic algorithms do well here, and they solve the environment pretty consistently. I try some variations to see how they affect performance.

In [1]:
# Display GIFs in Jupyter
from IPython.display import HTML

# OpenAI gym
import gym

# Import local script
import agents

# numpy
import numpy as np

# To speed up the algorithm
from multiprocessing import Pool
n_jobs = 4 # Set your number of cores here

I re-use the function from before.

In [2]:
def trial_agent(agent, trials=100, limit=1000):
    env = gym.make(agent.game)

    scores = []
    for i in range(trials):
        observation = env.reset()
        score = 0
        for t in range(limit):
            action = agent.predict(observation)
            observation, reward, done, info = env.step(action)
            if done:
                break
            score += reward
        scores.append(score)
        
    data_dict = {
        "agent" : agent, 
        "weights" : agent.w, 
        "pedigree" : agent.pedigree, 
        "minimum" : min(scores), 
        "maximum" : max(scores), 
        "mean" : sum(scores)/len(scores)
    }
    
    env.close()
    
    return data_dict

## The genetic algorithm

Below is my implementation of a genetic algorithm. I designed this by reading the Wikipedia article and talking to a few people. It's definitely not the best out there, but it works fine for cartpole.

When you keep the top agents from a generation, this is called [elitism](https://en.wikipedia.org/wiki/Genetic_algorithm#Elitism). It's a way to ensure that the next generation doesn't ruin itself with mutations and bad parenting. The "elite" from a round of testing stick around to maintain the status quo.

Next the parents are [selected](https://en.wikipedia.org/wiki/Selection_(genetic_algorithm%29) to create the offspring. This [genetic operation](https://en.wikipedia.org/wiki/Crossover_(genetic_algorithm%29) the main part of the algorithm. By mixing genes together, you're effectively searching the parameter space for a solution. Wikipedia uses a crossover method but I just randomly mix them. My agents mix their DNA by drawing genes from a hat, I guess.

[Mutations](https://en.wikipedia.org/wiki/Mutation_(genetic_algorithm%29) are rare, but they're a way of developing new genes that don't exist in the population. This should the agents escape from a local minimum.

In [3]:
def genetic_algorithm(results, old=5, new=95, n_parents=2, generations=25, mutation_rate=0.01, mutation_amount=0.5, order=1, max_score=499.0, cartpole=True):
    for round in range(generations):
        # Sort agents by score (fitness)
        top_scores = sorted(results, key=lambda x: x["mean"], reverse=True)

        # The survival of the fittest. Wikipedia calls this "elitism".
        # The top agents of a generation are carried over to the next
        survivors = top_scores[:old]

        # To start breeding new agents, I'll mix weights (genes)
        gene_pool = [i["weights"] for i in top_scores]
        pedigree_list = [i["pedigree"] for i in top_scores]
        genome_size = len(gene_pool[0])

        # Scores can be negative, so here I make them all positive
        # They also need to sum to 1 for random sampling
        min_score = min([i["mean"] for i in top_scores])
        sum_score = sum([i["mean"]+min_score for i in top_scores])
        probs = [(i["mean"]+min_score)/sum_score for i in top_scores]

        # For each new agent, randomly select parents
        # Higher-fitness agents are likelier to sire new agents
        children = []
        for birth in range(new):
            parents = np.random.choice(np.arange(len(gene_pool)), 
                             size=n_parents, 
                             replace=False, 
                             p=probs)

            # The offspring get a mix of each parent's weights
            # The weights (genes) are simply copied over
            mix = np.random.randint(0, high=n_parents, size=genome_size)

            weights = []
            pedigree = []
            for i in range(genome_size):
                weights.append(gene_pool[parents[mix[i]]][i])
                pedigree.append(pedigree_list[parents[mix[i]]][i])
                # A mutation happens rarely and adds a bit of noise to a gene
                if np.random.random(1) < mutation_rate:
                    weights[i] += float(np.random.normal(0, mutation_amount, 1))
                    pedigree[i] += "M"

            children.append({"weights" : weights, "pedigree" : pedigree})

        # Elitism: the top agents survive to fight another day
        new_agents = [i["agent"] for i in survivors]

        # The offspring are added it
        # With the pedigree variable their ancestors are tracked
        for child in children:
            new_agents.append(agents.LinearAgent(child["weights"], 
                pedigree=child["pedigree"],
                order=order,
                cartpole=cartpole))

        # Trial the agents using multiple CPU cores
        p = Pool(n_jobs)
        results = p.map(trial_agent, new_agents)
        p.close()
        
        results = sorted(results, key=lambda x: x["mean"], reverse=True)

        print(f"[{round+1:3}] Population average: {sum([i['mean'] for i in results])/len(results):5.1f}")
        print(f"[{round+1:3}] Best mean score:    {results[0]['mean']:5.1f}, Pedigree: {'-'.join(results[0]['pedigree'])}")
        print()
        
        # End early if maximum is reached
        if results[0]['mean'] >= max_score:
            print(f"[{round+1:3}] Best score reached, ending early")
            break
    return results

## First-order model

The process I use from now takes this form: I randomly generate an initial population, I breed them for a while, and I display the best agent.

Below you can see that the original population isn't that bad. It sometimes already solves the environment, but other times it fails miserably.

In [4]:
results = []

for a in range(25):
    results.append(trial_agent(agents.LinearAgent(None, id=a)))

winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_simple_test.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69b8a53fd0>, 'weights': array([-0.17569642, -0.17347816,  0.90116597,  0.38367552,  0.8680113 ]), 'pedigree': ['2', '2', '2', '2', '2'], 'minimum': 299.0, 'maximum': 350.0, 'mean': 322.66}


This runs the genetic algorithm. Each round you can see the population's average score/fitness. For the fittest agent you can see their mean score and their pedigree. Each agent of the original population is given an ID number, and these are how the genees are numbered. If there's a mutation, an M is appended to the ID number.

In [5]:
results = genetic_algorithm(results, generations=25)

[  1] Population average:  76.3
[  1] Best mean score:    332.9, Pedigree: 2-2-2-20-2

[  2] Population average: 172.3
[  2] Best mean score:    499.0, Pedigree: 5-2-2-2-2

[  2] Best score reached, ending early


It's very likely the scores will increase consistently each round. Usually the environment is solved within the 25 rounds.

In the example I ran, the cart runs off the screen but doesn't reach all the way within the 500 time steps. This counts as a win, even if isn't a very elegant one.

In [6]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_simple.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ad42ba90>, 'weights': [-0.054989080638153, -0.17347816176504915, 0.9011659658362716, 0.3836755217527761, 0.8680112986636497], 'pedigree': ['5', '2', '2', '2', '2'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}


## Second-order agent

Is there an advantage to making the agent's logistic regresion into a second order one? This would allow the agent to develop more sophisticated policies.

It doesn't seem to make much of a difference.

In [7]:
results = []

for a in range(25):
    results.append(trial_agent(agents.LinearAgent(None, order=2, id=a)))

winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_complex_test.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ad3eea20>, 'weights': array([ 0.03707171, -0.92788392, -0.21325485,  0.7412066 ,  0.91257579,
        0.67491734, -0.4100488 , -0.72374771, -0.12210238]), 'pedigree': ['23', '23', '23', '23', '23', '23', '23', '23', '23'], 'minimum': 42.0, 'maximum': 454.0, 'mean': 119.61}


In [8]:
results = genetic_algorithm(results, generations=25, order=2)

[  1] Population average:  25.8
[  1] Best mean score:    149.6, Pedigree: 16-16-16-23-16-16-16-16-23

[  2] Population average:  42.7
[  2] Best mean score:    180.4, Pedigree: 16-22-6-22-16-22-6-6-23

[  3] Population average:  78.0
[  3] Best mean score:    194.3, Pedigree: 23-16M-23-23M-23-16-23-16-23

[  4] Population average: 104.1
[  4] Best mean score:    278.2, Pedigree: 23-16-16-23M-23-24-24-2M-16

[  5] Population average: 134.2
[  5] Best mean score:    435.5, Pedigree: 16-22-6-23-16-22-23-6-23

[  6] Population average: 161.8
[  6] Best mean score:    430.3, Pedigree: 16-22-6-23-16-22-23-6-23

[  7] Population average: 183.9
[  7] Best mean score:    444.6, Pedigree: 16-22-6-23-16-22-23-6-23

[  8] Population average: 202.4
[  8] Best mean score:    498.4, Pedigree: 16-12-6-23-23-16-23M-12-3

[  9] Population average: 231.4
[  9] Best mean score:    498.2, Pedigree: 16-12-6-23M-23-2-23-12-5

[ 10] Population average: 275.0
[ 10] Best mean score:    498.4, Pedigree: 16-12-6

In [9]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_complex.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69b8a539b0>, 'weights': [-0.0886339218743244, -0.09345686626850158, 0.40273563111617805, 0.7412066030080693, 0.9125757881825356, -0.1901023066067098, -0.304465480356124, -0.35373297694590344, 0.9647327637102172], 'pedigree': ['16', '12', '6', '23', '23', '16', '23M', '12', '3'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}


## A bad initial gene pool?

With a good initial batch of agents, the genetic algorithm can just fine-tune these good guesses. But what about a bad batch? I'm curious to know if mutation play a bigger role here.

By selecting the bottom 10% of an initial population of 250, I get agents that topple the pole as quickly as they can. It's even hard to see it fall in the GIF.

This bad initial batch doesn't seem to overly harm the genetic algorithm. I try both no-mutations and high-mutations; both work well.

In [10]:
results = []

# Run 50 random agents
for a in range(250):
    results.append(trial_agent(agents.LinearAgent(None, id=a)))

# Select the bottom tenth
bottom_tenth = sorted(results, key=lambda x: x["mean"], reverse=True)[-25:]

winner = bottom_tenth[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('bad_genetic_simple_test.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69b852f550>, 'weights': array([-0.77408884,  0.07553019,  0.4379799 , -0.88718617, -0.77241714]), 'pedigree': ['193', '193', '193', '193', '193'], 'minimum': 7.0, 'maximum': 10.0, 'mean': 8.27}


### No mutation

In [11]:
results = genetic_algorithm(bottom_tenth, generations=25, mutation_rate=0.0)

[  1] Population average:   8.4
[  1] Best mean score:     10.6, Pedigree: 126-126-43-126-126

[  2] Population average:   8.7
[  2] Best mean score:     25.7, Pedigree: 51-51-99-51-126

[  3] Population average:  12.2
[  3] Best mean score:    158.7, Pedigree: 212-19-19-19-103

[  4] Population average:  23.7
[  4] Best mean score:    346.4, Pedigree: 212-99-19-19-189

[  5] Population average:  87.5
[  5] Best mean score:    359.5, Pedigree: 212-99-19-19-103

[  6] Population average: 180.6
[  6] Best mean score:    499.0, Pedigree: 212-51-19-126-103

[  6] Best score reached, ending early


In [12]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('bad_genetic_simple.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ac307518>, 'weights': [-0.020055077929563403, 0.036112456666890225, 0.28209796776440843, 0.39527133343592546, 0.6810513200652717], 'pedigree': ['212', '51', '19', '126', '103'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}


### High mutation

In [13]:
results = genetic_algorithm(bottom_tenth, generations=25, mutation_rate=0.1)

[  1] Population average:  10.1
[  1] Best mean score:    121.7, Pedigree: 51-19-19-19-51

[  2] Population average:  12.8
[  2] Best mean score:    128.2, Pedigree: 51-19-19-19-51

[  3] Population average:  25.5
[  3] Best mean score:    317.4, Pedigree: 51-186-3-19-51M

[  4] Population average:  79.8
[  4] Best mean score:    487.7, Pedigree: 51-70MM-3-103-51MM

[  5] Population average: 134.6
[  5] Best mean score:    496.7, Pedigree: 70MM-103-3-103-51MMM

[  6] Population average: 216.0
[  6] Best mean score:    496.7, Pedigree: 70MM-103-3-103-51MMM

[  7] Population average: 272.4
[  7] Best mean score:    499.0, Pedigree: 212-103-3-43M-51M

[  7] Best score reached, ending early


In [14]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('mutants_genetic_simple.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ac32d6a0>, 'weights': [-0.020055077929563403, -0.10335984368510953, 0.6440546417054149, 1.1004535196746557, 1.227039034630112], 'pedigree': ['212', '103', '3', '43M', '51M'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}


## More than two parents

[Wikipedia mentions](https://en.wikipedia.org/wiki/Genetic_algorithm#Genetic_operators) that having more than two parents is beneficial for the genetic algorithm.

In the example below, both cases reach the same solution. The 5-parent case reaches it faster.

In [15]:
results = []

for a in range(25):
    results.append(trial_agent(agents.LinearAgent(None, order=2, id=a)))

winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_parents_test.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ad3ea630>, 'weights': array([ 0.17117259, -0.42051824, -0.69305405,  0.58437648,  0.71336213,
       -0.4233984 , -0.19738188, -0.11602332, -0.66788128]), 'pedigree': ['20', '20', '20', '20', '20', '20', '20', '20', '20'], 'minimum': 29.0, 'maximum': 185.0, 'mean': 60.68}


### 2 parents

In [16]:
results = genetic_algorithm(results, generations=25)

[  1] Population average:  21.3
[  1] Best mean score:    158.6, Pedigree: 12-10-10-10-12-10-10M-12-10

[  2] Population average:  55.7
[  2] Best mean score:    493.9, Pedigree: 2-2-15M-4-15-23-23-23-15

[  3] Population average: 147.5
[  3] Best mean score:    499.0, Pedigree: 2-2-10-4-4-4-10M-12-2

[  3] Best score reached, ending early


In [17]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_2_parents.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ad3b2d30>, 'weights': [0.03015264316195765, -0.15945312975669057, 0.7567563891109443, 0.29837327774166544, 0.7965712055862131, -0.8293487438407883, -0.23911248392405504, 0.25129328180104715, -0.2488611639785323], 'pedigree': ['2', '2', '10', '4', '4', '4', '10M', '12', '2'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}


### 5 parents

In [18]:
results = genetic_algorithm(results, n_parents=5, generations=25)

[  1] Population average: 224.0
[  1] Best mean score:    499.0, Pedigree: 2-2-10-4-4-4-10M-12-2

[  1] Best score reached, ending early


In [19]:
winner = sorted(results, key=lambda x: x["mean"], reverse=True)[0]

print(winner)

HTML(f"<img src='{winner['agent'].render('genetic_5_parents.gif')}'>")

{'agent': <agents.LinearAgent object at 0x7f69ac3026a0>, 'weights': [0.03015264316195765, -0.15945312975669057, 0.7567563891109443, 0.29837327774166544, 0.7965712055862131, -0.8293487438407883, -0.23911248392405504, 0.25129328180104715, -0.2488611639785323], 'pedigree': ['2', '2', '10', '4', '4', '4', '10M', '12', '2'], 'minimum': 499.0, 'maximum': 499.0, 'mean': 499.0}
